In [2]:
import pandas as pd
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.document_loaders.dataframe import DataFrameLoader
from langchain.storage import LocalFileStore
from langchain.embeddings import CacheBackedEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [9]:
DATASET_PATH = "..\Amap-results_NJU-Gulou-3000m.csv"

<>:1: SyntaxWarning: invalid escape sequence '\A'
<>:1: SyntaxWarning: invalid escape sequence '\A'
C:\Users\Chrom\AppData\Local\Temp\ipykernel_13916\3351047562.py:1: SyntaxWarning: invalid escape sequence '\A'
  DATASET_PATH = "..\Amap-results_NJU-Gulou-3000m.csv"


In [ ]:
def get_documents(content_func=lambda row: row['name'] + '\n' + row['tag'],  # 把数据文档中每一行的餐厅名字和tag拆解出来的待定函数
                  source_func=lambda row: row['address'],  # 把数据文档中每一行的餐厅地址拆解出来的待定函数
                  metadata_fields=[]):  # 元数据字段列表，默认为空

    # 加载数据库，读取指定路径的 CSV 文件
    dataset_df = pd.read_csv(DATASET_PATH)
    
    # 删除重复数据，确保数据唯一性
    dataset_df.drop_duplicates(inplace=True)

    # 对数据中的每一行应用 content_func 函数，在数据中新增一列 page_content，将形如 name + '\n' + tag 的字符串作为文档内容
    dataset_df['page_content'] = dataset_df.apply(content_func, axis=1)
    
    # 对数据中的每一行应用 source_func 函数，在数据中新增一列 source，将形如 address 的字符串作为文档来源
    dataset_df['source'] = dataset_df.apply(source_func, axis=1)

    # 将 'page_content' 和 'source' 添加到元数据字段列表中，确保它们被包含
    metadata_fields = list(set(metadata_fields + ['page_content', 'source']))

    # 使用 DataFrameLoader 加载数据，指定 'page_content' 列作为文档内容
    loader = DataFrameLoader(dataset_df[metadata_fields], page_content_column='page_content')
    
    # 返回加载后的文档对象，所加载的内容为metadata_fields中指定的字段，其中以 'page_content' 列作为文档内容
    return loader.load()

In [10]:
def content_func(row) -> str:
  content_fields = ["name",
                    "address",
                    "type",
                    "tag",
                    "cost",
                    "rating",
                    "opentime_today",
                    "tel"]
  return '\n'.join(f"{key}={row[key]}" for key in content_fields if pd.notna(row[key]))

metadata_fields = ["location", "opentime_week"]

documents = get_documents(content_func, metadata_fields=metadata_fields)

page_content 专注于语义检索，metadata 专注于过滤和排序。
例如，location 字段不需要参与语义检索，但可以用于地理位置过滤。
同时问答系统是要根据数据结构来的，有什么数据，设计什么样的模型！

In [5]:
## 展示
print(documents[0].page_content)
print(documents[0].metadata)

name=巴蜀鱼花(南大店)
address=湖南路街道汉口路30号
type=餐饮服务;中餐厅;火锅店
rating=4.4
opentime_today=09:00-21:00
tel=15380870767
{'source': '湖南路街道汉口路30号', 'location': '118.779220,32.053685', 'opentime_week': '周一至周日 09:00-21:00'}


In [ ]:
# from dotenv import load_dotenv
# from langchain_openai import OpenAIEmbeddings
# import os

# #加载环境变量
# load_dotenv()
# #配置嵌入模型
# EMBEDDING_MODEL_NAME = "text-embedding-3-small"  
# embedding_model = OpenAIEmbeddings(
#     model=EMBEDDING_MODEL_NAME,
#     openai_api_key=os.getenv("OPENAI_API_KEY")
# )

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # 轻量级模型
    model_kwargs={"device": "cpu"},  # 可选 "cuda" 加速
    encode_kwargs={"normalize_embeddings": True})

In [10]:
result = embedding_model.embed_query("One sample query!")

In [11]:
import numpy as np
array = np.array(result)
print(f"embedding shape: {array.shape}\nembedding norm: {np.linalg.norm(array, ord=2)}")

embedding shape: (384,)
embedding norm: 1.0000000478534594


## FAISS数据库

In [12]:
#存储向量索引
FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine"
FAISS_INDEX_NAME = "index"
FAISS_DISTANCE_STRATEGY_COSINE = "COSINE_DISTANCE"

在代码 FAISS_REVIEWS_PATH_COSINE = "faiss_index_cosine" 中，"faiss_index_cosine" 是 FAISS 向量索引的本地存储路径，保存的是经过向量化处理后的文档索引数据。以下是详细解释：

存储的具体内容
当调用 vector_db.save_local(FAISS_REVIEWS_PATH_COSINE) 时，会在该路径下生成以下文件：

index.faiss
二进制文件，存储向量索引的核心数据（包括向量数据、索引结构等）。

index.pkl（可选）
存储元数据（如文档的原始文本、ID等，需通过 LangChain 额外配置）。

这些文件共同构成一个完整的可复用的向量数据库。

In [13]:
def get_vector_database(documents, embedding_model, distance_strategy):

  vector_database = FAISS.from_documents(
      documents, embedding_model,
      distance_strategy= distance_strategy
      )
  return vector_database

In [14]:
#分批次处理文档，主要考虑调用openai时的api限制
import time
doclen = len(documents)
for batch in range(doclen//100 + 1):
    docs = documents[batch*100:(batch+1)*100]
    if batch ==0:
        vector_db = get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE)
    else:

        vector_db.merge_from(get_vector_database(docs, embedding_model, FAISS_DISTANCE_STRATEGY_COSINE))
    time.sleep(10) # Sleep for 10 seconds to avoid hitting rate limits

In [15]:
#储存向量库
vector_db.save_local(folder_path=FAISS_REVIEWS_PATH_COSINE, index_name=FAISS_INDEX_NAME)

In [16]:
#加载向量库,允许反序列化
vector_db = FAISS.load_local(folder_path=FAISS_REVIEWS_PATH_COSINE,
                             embeddings=embedding_model,
                             index_name=FAISS_INDEX_NAME,
                             allow_dangerous_deserialization=True)

In [17]:
docs = vector_db.similarity_search("给我找一下汉口路附近的混沌店", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='name=西安特色面馆(汉口路店)
address=汉口路47号01幢一楼
type=餐饮服务;中餐厅;中餐厅
tag=肉夹馍
cost=13.0
rating=4.0
opentime_today=07:20-21:30
tel=18114808698' metadata={'source': '汉口路47号01幢一楼', 'location': '118.779220,32.053475', 'opentime_week': '每天07:20-21:30'}

page_content='name=鱼你在一起(恒基中心公寓店)
address=丹凤街29号大润发一层
type=餐饮服务;餐饮相关场所;餐饮相关
rating=4.2
opentime_today=09:00-21:00
tel=13770909712' metadata={'source': '丹凤街29号大润发一层', 'location': '118.786668,32.054497', 'opentime_week': '周一至周日 09:00-21:00'}

page_content='name=沙县小吃(汉口路店)
address=汉口路73号
type=餐饮服务;中餐厅;特色/地方风味餐厅
cost=13.0
rating=4.4
opentime_today=10:00-21:00
tel=18752023233;19941534156' metadata={'source': '汉口路73号', 'location': '118.778214,32.053989', 'opentime_week': '周一至周日 10:00-21:00'}

page_content='name=箪食记(汉口路店)
address=汉口路75-2号
type=餐饮服务;中餐厅;中餐厅
cost=31.0
rating=4.2
tel=13951883009;15651837225' metadata={'source': '汉口路75-2号', 'location': '118.777185,32.053960', 'opentime_week': '周一至周六 09:00-20:00'}

page_content='name=鱼塘鲜专业鱼馆(汉口路店)
addre

In [18]:
docs = vector_db.similarity_search("我想吃菜煎饼", k = 5)
for doc in docs:
    print(doc, end="\n\n")

page_content='name=鱼你在一起(恒基中心公寓店)
address=丹凤街29号大润发一层
type=餐饮服务;餐饮相关场所;餐饮相关
rating=4.2
opentime_today=09:00-21:00
tel=13770909712' metadata={'source': '丹凤街29号大润发一层', 'location': '118.786668,32.054497', 'opentime_week': '周一至周日 09:00-21:00'}

page_content='name=福桔家庭厨房
address=汉口路42号
type=餐饮服务;中餐厅;中餐厅
tag=寿喜锅,糖醋排骨
rating=4.7
opentime_today=11:00-13:30 17:00-20:30
tel=19961882001' metadata={'source': '汉口路42号', 'location': '118.778479,32.053663', 'opentime_week': '周日 11:00-14:30,17:00-20:30；周六 11:00-20:30；周一至周五 11:00-13:30,17:00-20:30'}

page_content='name=美鸽记·中山石岐乳鸽
address=青岛路33-3号
type=餐饮服务;中餐厅;中餐厅
tag=红米肠
rating=4.6
opentime_today=10:00-14:00 16:00-22:00
tel=18752079369' metadata={'source': '青岛路33-3号', 'location': '118.778138,32.052878', 'opentime_week': '周一至周日 10:00-14:00，16:00-22:00'}

page_content='name=同廣鸣港式烧腊(南京大学店)
address=汉口路47-2号
type=餐饮服务;中餐厅;中餐厅
tag=港式烧腊
rating=4.4
opentime_today=10:00-22:00
tel=13305154735' metadata={'source': '汉口路47-2号', 'location': '118.779262,32.053522', '

## 导入大语言模型

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

In [23]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()
if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")
from langchain.chat_models import init_chat_model
llm = init_chat_model("gpt-4o-mini", model_provider="openai")


In [24]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
messages = [
    SystemMessage("Translate the following from English into French"),
    HumanMessage("hi!"),
]
llm.invoke(messages)

AIMessage(content='Salut !', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_dbaca60df0', 'id': 'chatcmpl-BTkhUGgaGG8zve7xTlpMbHtCVAWYi', 'finish_reason': 'stop', 'logprobs': None}, id='run-6f9e678e-ac6e-4e05-ad69-d72503cfc5d2-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## LangChain pipeline

In [25]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import FAISS
from langchain.schema.runnable import RunnablePassthrough

In [ ]:
review_template_str = """
你的工作是使用高德地图上的餐馆和酒吧评论来帮助人们找到吃饭或喝酒的最佳地点。
使用以下信息和评论回答问题。如果上下文不是关于餐馆的，
然后请告诉用户，您只能提供帮助并回答与餐厅相关的问题。
如果你根据上下文不知道答案，就说你不知道。答案上下文
{context}
"""

system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_template_str
    )
)

human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(input_variables=["question"], template="{question}")
)
messages = [system_prompt, human_prompt]

review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"], messages=messages
)

reviews_retriever = vector_db.as_retriever(search_kwargs={'k': 20,})

review_chain = (
    {"context": reviews_retriever, "question": RunnablePassthrough()}
    | review_prompt_template
    | llm
    | StrOutputParser()
)

## 案例应用

In [28]:
question = """汉口路哪里可以吃到四川的面"""
print(review_chain.invoke(question))

在汉口路，您可以尝试以下餐馆，提供四川菜和面食：

1. **四川宜宾燃面**
   - 地址：汉口路48号
   - 评分：4.7
   - 营业时间：周一至周日 09:00-20:30
   - 电话：13072527682; 17326125923

2. **兰州拉面刀削面(汉口路店)**
   - 地址：汉口路36号
   - 评分：4.5
   - 营业时间：周一至周日 09:00-21:00
   - 电话：17327791285

3. **西安特色面馆(汉口路店)**
   - 地址：汉口路47号01幢一楼
   - 评分：4.0
   - 营业时间：每天 07:20-21:30
   - 电话：18114808698

这些地方都可以尝到美味的四川面。希望您能享用美食！


In [31]:
question = """我想吃肯德基"""
print(review_chain.invoke(question))

你可以去以下两个肯德基：

1. **肯德基(珠江路地铁店)**
   - 地址：中山路221号负一层101至106室115至119室
   - 营业时间：周一至周日 06:00-21:00
   - 电话：025-85601286
   - 评分：4.3
   - 费用：约21元

2. **肯德基(学府店)**
   - 地址：丹凤街39号金润发购物中心1层
   - 营业时间：周一至周日 10:00-23:00
   - 电话：025-83222982
   - 评分：4.8
   - 费用：约44元

3. **肯德基(广州店)**
   - 地址：广州路104号
   - 营业时间：24小时营业
   - 电话：025-86632902
   - 评分：4.6
   - 费用：约41元

你可以根据自己的位置和时间选择最合适的肯德基。
